# Premiers pas avec MPI

## Qu'est-ce que MPI?

* **Message Passing Interface** (interface de transmission de messages)
* Ce n'est **pas** un langage de programmation
* C'est une **bibliothèque de fonctions** qu'on peut
  appeler à partir du Fortran, du C ou encore du C++
* C'est une [norme officielle](https://www.mpi-forum.org/docs/)
  (de MPI 1.0 en 1994 à
  [MPI 4.0](https://www.mpi-forum.org/docs/mpi-4.1/mpi41-report.pdf)
  depuis novembre 2023)
* Hors norme, il existe des interfaces en d'autres langages
  (Python, R, Perl, Java)

## Structure d'un code MPI

* Dans un programme MPI, on doit d'abord appeler la fonction
  `MPI_Init()`
* Avant de terminer l'exécution, on appelle la fonction
  `MPI_Finalize()`
* Tous les autres appels à MPI doivent se
  faire entre les deux nommés ci-dessus
* Pour une fin d'exécution prématurée, un
  processus appelle la fonction `MPI_Abort()`

```C
MPI_init()

// Code parallèle

// En cas d’erreur
{
    MPI_Abort()
}

MPI_Finalize()
```

### Programme MPI de base

En C :

```C
#include "mpi.h"

int main(int argc, char **argv)
{
    int ierr;
    ierr = MPI_Init(&argc, &argv);

    /* On insère le reste du programme ici. */

    ierr = MPI_Finalize();
}
```

En Python :

```Python
#!/usr/bin/env python

from mpi4py import MPI  # MPI.Init() implicite

def main():
    """
    Programme principal
    """

    MPI.Finalize()

if __name__ == '__main__':
    main()
```

## Parallélisation explicite

* C'est la responsabilité du programmeur de dire à chaque processus
  ce qu'il doit faire selon son _rang_ dans la liste des processus
* Chaque processus échange avec les
  autres les données dont ils ont besoin
* Un code source appelant la bibliothèque MPI utilise donc (en
  général) le concept de programme unique travaillant sur des
  données multiples (_Single Program, Multiple Data_ ou SPMD)

### Communicateurs

* Un communicateur désigne un ***groupe de
  processus*** pouvant communiquer ensemble
* La majorité des fonctions MPI prennent comme argument un
  communicateur pour savoir sur quel groupe de processus agir
* Un communicateur initial appelé
  `MPI_COMM_WORLD` englobe tous les processus
* On peut en créer d'autres au besoin

![Communicateurs (groupes)](images/communicators.png)

### Nombre de processus et un rang unique

* Dans l'optique SPMD, un processus doit savoir
  le **nombre de processus total** (`size`) et
  **son rang** (`rank`) dans le communicateur
* Le rang est un identifiant entier unique dont
  la valeur est de `0` à `size-1`, inclusivement

En C :

```C
int MPI_Comm_size(MPI_Comm comm, int *size)
int MPI_Comm_rank(MPI_Comm comm, int *rank)
```

En Python :

```Python
rank = MPI.COMM_WORLD.Get_rank()
size = MPI.COMM_WORLD.Get_size()
```

#### Exemple en C - `demos/bonjour.c`

```C
#include <stdio.h>
#include <mpi.h>

int main(int argc, char *argv[])
{
    int p, np;

    MPI_Init(&argc, &argv);

    MPI_Comm_rank(MPI_COMM_WORLD, &p);
    MPI_Comm_size(MPI_COMM_WORLD, &np);
    printf("Ici le processus %d de %d\n", p, np);

    MPI_Finalize();
    return 0;
}
```

#### Exemple en Python - `demos/bonjour.py`

```Python
#!/usr/bin/env python

from mpi4py import MPI  # MPI.Init() implicite

def main():
    """
    Programme principal
    """

    p = MPI.COMM_WORLD.Get_rank()
    np = MPI.COMM_WORLD.Get_size()
    print(f'Ici le processus {p} de {np}')

    MPI.Finalize()

if __name__ == '__main__':
    main()
```

## Environnement requis pour utiliser MPI

En C :

* Un module permettant de compiler et d’exécuter un
  programme MPI est chargé par défaut (`openmpi/4.1.5`)
* Pour voir tous les modules chargés :
```Bash
module list
```
* Pour voir les différentes versions disponibles de Open MPI :
```Bash
module spider openmpi
```
* Pour voir les dépendances possibles :
```Bash
module spider openmpi/4.1.5
```

En Python :

* Un module `mpi4py` est requis, mais n'est pas chargé par défaut :
```Bash
module spider mpi4py
module spider mpi4py/3.1.4
module load mpi4py/3.1.4  # python/3.11.5 est chargé automatiquement
```

### Compilation d'un programme MPI

En C :

* Lorsque l’environnement est adéquatement configuré pour la
  version désirée de MPI, on a accès aux commandes `mpif90`,
  `mpicc` et `mpicxx` que l'on peut utiliser à la place des
  compilateurs Fortran, C et C++, respectivement. Par exemple :
```Bash
mpicc -o programme source.c
```
* Ces commandes trouvent les fichiers d'inclusion et les
  bibliothèques ou les modules Fortran nécessaires à MPI

En Python :

* La compilation se fait lors de l'exécution du script.

### Lancement d'un programme MPI

Le lancement d'un programme MPI se fait typiquement à l'aide d'une
commande souvent appelée `mpirun` ou `mpiexec`. Par exemple :

```Bash
mpiexec -n 120 ./programme
# ou
mpiexec -n 120 python script.py
```

Notes :

* Chacune des commandes ci-dessus lancerait 120 processus identiques,
  chacun exécutant le `programme` ou `python` sur un processeur
* Avec l’ordonnanceur [Slurm](https://slurm.schedmd.com/),
  nous allons plutôt utiliser la commande `srun`

#### Exemple d’exécution

```Bash
$ srun -n 10 hostname
compute-node2
compute-node2
compute-node2
compute-node2
compute-node2
compute-node1
compute-node1
compute-node1
compute-node1
compute-node1

$
```

### Exercice #1 - Premier lancement

1. Validez les modules et chargez les modules manquants, au besoin :
```Bash
module list
module load ...
```
2. Dans le répertoire `demos/`, compilez le code
   `bonjour.c` ou localisez le fichier `bonjour.py`
3. Avec `srun`, lancez 4 processus en parallèle
   avec l'exécutable compilé ou avec `python`

## Partage du travail

Maintenant que chaque processus connaît
sa place dans le communicateur, il peut :

* **effectuer la partie du travail** correspondant
  à son numéro de processus (le _rang_),
* **communiquer aux autres** processus le résultat de son travail,
* **recevoir des autres** le résultat de leur travail.